<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab4_inferring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferring
ใน notebook นี้เราจะได้ทดลองการใช้ LLM ในการอนุมาน หัวข้อ อารมณ์ และความรู้สึก จากการ review สินค้า และจากบทความข่าว

## Setup

In [ ]:
# !pip -q install python-dotenv
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
#========= VSCode for Groq =====================================================================
# import os
# from dotenv import load_dotenv
# load_dotenv()
# secret_key = os.environ.get("GROQ_API_KEY")
#======== google.colab for Groq ================================================================
from google.colab import userdata
secret_key = userdata.get('GROQ_API_KEY')

#===== create Groq object=================
from groq import Groq
clientGroq = Groq(
    api_key= secret_key,
)

In [ ]:
def get_completion(prompt, model= "llama3-70b-8192", clientGroq = clientGroq):
    messages = [{"role": "user", "content": prompt}]
    response = clientGroq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 4000,
        top_p = 0.5
    )
    return response.choices[0].message.content

## Product review text
ข้อมูลจากเวปพันทิพย์ https://pantip.com/topic/33261003

In [ ]:
# Positive Review for a VIVO smartphone
# from Pantip https://pantip.com/topic/33261003
#
lamp_review = """
เมื่อ 3 ปีที่แล้วก้อลังเลแบบนี้แหละค่ะ พอดีช่วงนั้นต้องเดินทางไปต่างประเทศและอยากเก็บรูปสวยๆให้มากที่สุด
แต่ไม่อยากซื้อกล้องถ่ายรูป เลยมอง smartphoneแทนเพราะส่วนใหญ่เน้นถ่ายรูปกับ social media
ตอนไปที่ร้านตั้งใจไปซื้อซัมซุง แต่ พนง ขายแนะนำ X-shot หลังจากเทียบสเปค เทียบราคาจึงเลือก X-shot ใช้ 3 ปี
ไม่เคยซ่อม มีรวนครั้งนึงเพราะรูปเยอะมาก แบตก้อถือได้ว่า 70% สำหรับเครื่อง 3 ปีก้อถือว่าเยี่ยม
ตอนเดือน พ.ค. ปีนี้เพิ่งถอย V5Plus มา Xshot ก้อยังใช้อยู่เพราะไม่อยากขาย ยังใช้งานได้ดีค่ะ
บอกได้เลยว่า Vivo ตอบโจทย์ lifestyle ที่สุด ตอนนี้เวลาใครมาถามเรื่อง smartphone
แนะนำ Vivo แน่นอนค่ะ ตอนนี้ขายของออนไลน์ ใช้แค่ smartphone ก้อได้รูปสวยๆ ก้อลงสินค้าได้แล้วค่ะ
(ปล.ที่ซื้อ v5plus เพราะต้องไปต่างประเทศอีกครั้ง แล้วกลัวว่า xshot
จะมีปัญหาเพราะตกจากที่สูงบ่อยมาก เล่นเกมหนักมาก สรุปคือคิดมากไปเอง ตอนนี้เลยใช้คู่กันทั้งสองเครื่องเลยค่ะ)

ถ้า lifestyle เน้นถ่ายรูปกับ social บอกได้เลยว่า vivo ตอบโจทย์ที่สุดค่ะ
"""

In [ ]:
# Negative Review for a VIVO smartphone
# from Pantip https://pantip.com/topic/33261003
#
lamp_review = """
ซื้อ vivo v7 มาหมดรับประกันแค่ 7 วัน เครื่องรับสัญณาณไม่ได้แล้ว
ที่แรกคิดว่าเป็นเพราะซิมที่ไหนได้เป็นที่เครื่องรับสัญณาณเสีย เสียเงินยังไม่พอ ยังหัวเสียอีก
ทางร้านไม่รับผิดชอบอะไรเลย บอกแต่ว่าเขาศูนย์เถอะคับ
ผมกะไม่รู้ว่าเกิดจากอะไรเหมือนกัน เราซื้อเครื่องจาก ไอทีสแควจากหลักสี่
พนักงานขายคนที่ขายหายหัวไปละ มีคนใหม่มาแทนไม่รู้เรื่องออะไรเลย
ใครที่คิดจะซื้อ V9 ต้องระวังให้ดี
สุดท้ายก็ต้องเปลี่ยนมาใช้ซัมซุงเหมือนเดิม ปัญหาน้อยหน่อย
ราคาไม่ต่างกัน แต่คุณภาพต่ำ 10 คิดว่าจะดี แต่ที่ไหนได้ห่วยแตก
"""

## Sentiment (positive/negative)

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple ticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of this product review is NEGATIVE.

Here's a breakdown of the review:

* The reviewer expresses frustration and disappointment with their Vivo V7 phone, which stopped working after just 7 days of purchase.
* They mention that the store didn't take responsibility for the issue and instead blamed the phone's reception problem on the SIM card.
* The reviewer also complains about the poor customer service, saying that the salesperson who sold them the phone was no longer available and the new staff didn't know what was going on.
* They warn others who are considering buying the Vivo V9 to be cautious.
* The reviewer eventually switched back to using a Samsung phone, implying that it's a better option.
* The reviewer uses strong negative language, such as "ห่วยแตก" (which roughly translates to "utterly useless" or "complete garbage"), to express their dissatisfaction with the Vivo phone.

Overall, the reviewer's experience with the Vivo V7 was extremely poor, and they would n

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple ticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Negative


## Identify types of emotions

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Here is the list of emotions expressed by the writer of the review:

angry, frustrated, disappointed, annoyed, bitter


## Identify anger

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Yes


## Extract product and company name from customer reviews

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Here is the response in JSON format:

```
{
  "Item": "vivo v7",
  "Brand": "vivo"
}
```


## Doing multiple tasks at once

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Here is the response in JSON format:

```
{
  "Sentiment": "negative",
  "Anger": true,
  "Item": "vivo v7",
  "Brand": "vivo"
}
```


## Inferring topics
ข่าวจากเวป GISDA https://www.gistda.or.th/news_view.php?n_id=7569&lang=TH

In [ ]:
story = """
14 มีนาคม 2567 กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกรรม (อว.)
โดยสำนักงานพัฒนาเทคโนโลยีอวกาศและภูมิสารสนเทศ (องค์การมหาชน) หรือ GISTDA
ร่วมลงนามความร่วมมือกับ องค์การบริหารการบินและอวกาศแห่งชาติ สหรัฐอเมริกา หรือ NASA
ภายใต้โครงการ Airborne and Satellite Investigation of Asian Air Quality (ASIA-AQ)
เพื่อร่วมกันศึกษาคุณภาพอากาศในประเทศไทย โดยมี นางสาวศุภมาส อิศรภักดี รัฐมนตรีว่าการกระทรวง อว.
ร่วมเป็นสักขีพยานในการลงนามครั้งนี้ ณ โรงแรมเรเนซองส์ พัทยา รีสอร์ท แอนด์ สปา จังหวัดชลบุรี
.
ความร่วมมือระหว่าง NASA และ GISTDA เกิดขึ้นอย่างเป็นรูปธรรมตามแนวทางของกระทรวง อว.
ที่ต้องการผลักดันนโยบายให้สามารถช่วยเหลือสังคมได้อย่างแท้จริง
นับเป็นความก้าวหน้าทางด้านวิทยาศาสตร์และเทคโนโลยีของประเทศที่สะท้อนถึงความสำเร็จ
จากความทุ่มเทที่ต้องการใช้เครื่องมือทางวิทยาศาตร์ในการแก้ไขปัญหาในระดับประเทศที่กำลัง
เผชิญอยู่ในชีวิตประจำวันโดยเฉพาะด้านสิ่งแวดล้อมซึ่งทุกวันนี้มลพิษทางอากาศนับว่ามีความรุนแรงเพิ่มมากขึ้น
ภายใต้ความร่วมมือนี้จะมีการใช้ประโยชน์ที่ได้จากการสำรวจทางอากาศ การสำรวจด้วยดาวเทียม
รวมถึงภาคพื้นดินเพื่อนำมาบูรณาการในการแก้ไขปัญหา
การจัดการและลดมลพิษทางอากาศเพื่อคุณภาพชีวิตที่ดีของคนไทยต่อไป
ตามนโยบายของรัฐบาลและกระทรวง อว. ที่ได้วางไว้

โครงการ Airborne and Satellite Investigation of Asian Air Quality
เป็นโครงการที่ริเริ่มโดย NASA เพื่อร่วมมือกับหน่วยงานต่างๆ ศึกษาปัญหาคุณภาพอากาศในเอเซีย โดย NASA
จะทำงานร่วมกับหน่วยงานในภูมิภาคเพื่อรวบรวมข้อมูลสำหรับการศึกษาด้านอากาศโดยใช้เครื่องบิน
สถานีภาคพื้นดิน และดาวเทียมสำรวจ ซึ่งโครงการนี้ NASA มีความร่วมมือกับ พันธมิตรในเอเชีย 5 ราย
ได้แก่ สาธารณรัฐเกาหลี ไต้หวัน ฟิลิปปินส์ มาเลเซีย และประเทศไทย
โดยมี GISTDA ทำหน้าที่เป็นหน่วยงานหลักในการจัดตั้งทีมร่วมกับนักวิทยาศาสตร์ไทยเพื่อทำงานร่วมกับ NASA
โดยทีมวิชาการประกอบด้วย กรมควบคุมมลพิษ มหาวิทยาลัยศิลปากร
และสถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง
และในการบินเพื่อเก็บข้อมูลอากาศในครั้งนี้
ได้รับการสนับสนุนการดำเนินการจากกระทรวงกลาโหมและหน่วยงานทางทหารที่เกี่ยวข้อง
(กองทัพอากาศ กองทัพเรือ กรมแผนที่ทหาร), บริษัท วิทยุการบินแห่งประเทศไทย จำกัด
และสำนักงานการบินพลเรือนแห่งประเทศไทย (CAAT)
.
นอกจากนี้ยังมีกิจกรรมอื่นๆ ที่ NASA ร่วมกับสถาบันส่งเสริมการสอนวิทยาศาสตร์และเทคโนโลยี
และ GISTDA จัดขึ้นภายใต้โครงการนี้ เช่น การอบรมเชิงปฏิบัติด้านอวกาศและดาวเทียมให้กับนิสิตนักศึกษา
ภายใต้โครงการ GLOBE ณ Space Inspirium อุทยานรังสรรค์นวัตกรรมอวกาศ อำเภอศรีราชา จังหวัดชลบุรี
กิจกรรมการแบ่งปันและแลกเปลี่ยนประสบการณ์ทำงานจากนักบินและนักวิทยาศาสตร์ NASA กับนักวิทยาศาสตร์ไทย
รวมถึงกิจกรรมอื่นๆ อีกมากมายที่เกี่ยวข้องกับการถ่ายทอดองค์ความรู้ในการใช้เทคโนโลยีอวกาศในการศึกษา
และสำรวจมลพิษทางอากาศ ข้อมูลที่ได้จากการสำรวจคุณภาพอากาศในประเทศไทย
ช่วยให้เข้าใจและเข้าถึงปัจจัยที่ใช้ในการควบคุมคุณภาพอากาศในพื้นที่ต่างๆ
เพื่อนำมาใช้ในการติดตามสถานการณ์การประเมินการปล่อยแก๊สจากแหล่งต่างๆ
ปรากฏการณ์ทางเคมี รวมถึงผลกระทบจากด้านการขนส่งประเภทต่างๆ
โดยผ่านการสร้างแบบจำลองที่หลากหลายเพื่อใช้สำหรับการออกแบบนโยบาย
ในการบริหารจัดการมลพิษทางอากาศในประเทศไทยและภูมิภาคเอเชียต่อไป ผู้อำนวยการ GISTDA กล่าว
"""

## Infer 5 topics

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple ticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Here are the five topics being discussed in the text, formatted as a list of items separated by commas:

Air Quality, Space Technology, Environmental Issues, Scientific Research, International Cooperation


In [ ]:
topic_list = [
    "nasa", "air quality", "science", "cooperation",
    "pollution", "international", "higher education",
    "local government", "engineering",
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\
for example
nasa : 1

List of topics: {", ".join(topic_list)}

Format your response as a JSON object.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Here is the list of topics with 0 or 1 indicating whether each topic is present in the text:

```
{
  "nasa": 1,
  "air quality": 1,
  "science": 1,
  "cooperation": 1,
  "pollution": 1,
  "international": 1,
  "higher education": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 0,
  "federal government": 0
}
```

Note that some topics may be mentioned indirectly or in a related context, but I've only marked them as 1 if they are explicitly mentioned in the text.


In [ ]:
import json
def extract_json_part(text):
  # Find the start and end of the JSON object
  start = text.find('{')
  end = text.rfind('}') + 1

  # Extract the JSON object as a string
  json_string = text[start:end]

  # Attempt to parse the JSON string
  try:
    json_object = json.loads(json_string)
    return json_object
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    return None


In [ ]:
extracted_json = extract_json_part(response)
print(extracted_json)

{'nasa': 1, 'air quality': 1, 'science': 1, 'cooperation': 1, 'pollution': 1, 'international': 1, 'higher education': 1, 'local government': 0, 'engineering': 0, 'employee satisfaction': 0, 'federal government': 0}


In [ ]:
if extracted_json['nasa'] == 1:
    print("ALERT: News related to NASA")

ALERT: News related to NASA


In [ ]:
if extracted_json['local government'] ==1:
    print("ALERT: News related to local government")
else:
    print("No news related to local government")

No news related to local government


## Try experimenting on your own!